In [57]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,precision_score
import joblib
import gradio as gr
import pandas as pd
import joblib
import numpy as np


In [58]:
dataset=pd.read_csv('/content/Voting_Intention_Prediction.csv')

In [59]:
dataset.sample(10)

,Voter_ID,Age,Gender,Income,Education,Region,Political_Affiliation,Campaign_Engagement,Social_Media_Usage,Voting_Intention
255,a0a70c65-4184-4ee8-907b-94b63979ec8d,46,Female,64086.0,PhD,Urban,Conservative,1,2,0
659,9bb15a69-47e8-4d4f-9cd6-f12d910d90f5,42,Female,131691.0,High School,Urban,Conservative,1,2,0
352,8e039ad0-1755-4d36-b639-f94d43a538d7,42,Female,140747.0,Masters,Urban,Liberal,0,4,0
471,e2f78759-6ed1-4ebf-bfa9-6c463dfe0c90,29,Female,56043.0,Bachelors,Rural,Liberal,0,5,0
759,78c87107-0458-4df1-be6f-f3be708fbe1c,31,Female,74694.0,Masters,Urban,Liberal,1,2,0
279,9877bed6-a6a2-4d13-8bfb-930ee0049207,76,Male,25411.0,Masters,Rural,Liberal,1,3,0
544,dea217a8-4062-49aa-bbb4-9c86cd040baf,26,Male,62113.0,Bachelors,Rural,Independent,0,4,1
791,d37e1bc0-6c21-4e79-98b3-86be7392e225,62,Female,52266.0,Masters,Urban,Independent,1,5,1
732,9ce5ee8e-b8fe-4780-9978-b09f3d23f864,21,Female,NaN,High School,Urban,Independent,0,5,0
557,73bf3180-018d-4f52-a799-5f7807fc3041,18,Male,61626.0,Bachelors,Rural,Independent,1,1,1


In [60]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 910 entries, 0 to 909
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Voter_ID               910 non-null    object 
 1   Age                    910 non-null    int64  
 2   Gender                 850 non-null    object 
 3   Income                 850 non-null    float64
 4   Education              855 non-null    object 
 5   Region                 910 non-null    object 
 6   Political_Affiliation  910 non-null    object 
 7   Campaign_Engagement    910 non-null    int64  
 8   Social_Media_Usage     910 non-null    int64  
 9   Voting_Intention       910 non-null    int64  
dtypes: float64(1), int64(4), object(5)
memory usage: 71.2+ KB


In [61]:
imputer=SimpleImputer(strategy='most_frequent')

In [62]:
dataset[['Gender','Income','Education']]=imputer.fit_transform(dataset[['Gender','Income','Education']])

In [63]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 910 entries, 0 to 909
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Voter_ID               910 non-null    object
 1   Age                    910 non-null    int64 
 2   Gender                 910 non-null    object
 3   Income                 910 non-null    object
 4   Education              910 non-null    object
 5   Region                 910 non-null    object
 6   Political_Affiliation  910 non-null    object
 7   Campaign_Engagement    910 non-null    int64 
 8   Social_Media_Usage     910 non-null    int64 
 9   Voting_Intention       910 non-null    int64 
dtypes: int64(4), object(6)
memory usage: 71.2+ KB


In [64]:
for col in['Income']:dataset[col]=pd.to_numeric(dataset[col],errors='coerce')

In [65]:
scaler=MinMaxScaler(feature_range=(0,1))

In [66]:
dataset[['Age','Income','Social_Media_Usage']]=scaler.fit_transform(dataset[['Age','Income','Social_Media_Usage']])

In [67]:
dataset.head()

,Voter_ID,Age,Gender,Income,Education,Region,Political_Affiliation,Campaign_Engagement,Social_Media_Usage,Voting_Intention
0,f154e23d-d11c-48e1-93f1-f622c0a93222,0.032787,Female,0.027595,Masters,Urban,Independent,0,0.25,1
1,e1547582-69a5-4e3e-b5e2-cba6fac568f7,0.754098,Male,0.470945,Bachelors,Urban,Independent,1,0.50,1
2,e01159d7-c5d3-4211-a5fa-123d70e17977,0.819672,Female,0.853105,PhD,Urban,Independent,1,0.25,1
3,57c49342-8335-410c-b77d-025e637ce22f,0.836066,Male,0.107162,Bachelors,Rural,Conservative,1,0.75,1
4,5cb55cfc-3304-4e85-a2a3-2ea3bb146153,0.688525,Male,0.523825,PhD,Rural,Conservative,1,0.50,1


In [68]:
encoder=OneHotEncoder()
columns_to_encode=['Gender','Education','Region','Political_Affiliation']
encoded_data = encoder.fit_transform(dataset[columns_to_encode])
# Convert the sparse matrix to a DataFrame before concatenation
encoded_data_df = pd.DataFrame(encoded_data.toarray(), columns=encoder.get_feature_names_out(columns_to_encode), index=dataset.index)
dataset = pd.concat([dataset, encoded_data_df], axis=1)
dataset=dataset.drop(columns=columns_to_encode)

In [69]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 910 entries, 0 to 909
Data columns (total 17 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Voter_ID                            910 non-null    object 
 1   Age                                 910 non-null    float64
 2   Income                              910 non-null    float64
 3   Campaign_Engagement                 910 non-null    int64  
 4   Social_Media_Usage                  910 non-null    float64
 5   Voting_Intention                    910 non-null    int64  
 6   Gender_Female                       910 non-null    float64
 7   Gender_Male                         910 non-null    float64
 8   Education_Bachelors                 910 non-null    float64
 9   Education_High School               910 non-null    float64
 10  Education_Masters                   910 non-null    float64
 11  Education_PhD                       910 non-n

In [70]:
dataset.head()

,Voter_ID,Age,Income,Campaign_Engagement,Social_Media_Usage,Voting_Intention,Gender_Female,Gender_Male,Education_Bachelors,Education_High School,Education_Masters,Education_PhD,Region_Rural,Region_Urban,Political_Affiliation_Conservative,Political_Affiliation_Independent,Political_Affiliation_Liberal
0,f154e23d-d11c-48e1-93f1-f622c0a93222,0.032787,0.027595,0,0.25,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1,e1547582-69a5-4e3e-b5e2-cba6fac568f7,0.754098,0.470945,1,0.50,1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,e01159d7-c5d3-4211-a5fa-123d70e17977,0.819672,0.853105,1,0.25,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,57c49342-8335-410c-b77d-025e637ce22f,0.836066,0.107162,1,0.75,1,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,5cb55cfc-3304-4e85-a2a3-2ea3bb146153,0.688525,0.523825,1,0.50,1,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0


In [71]:
dataset.drop(['Voter_ID'],axis=1,inplace=True)

In [72]:
dataset.head()

,Age,Income,Campaign_Engagement,Social_Media_Usage,Voting_Intention,Gender_Female,Gender_Male,Education_Bachelors,Education_High School,Education_Masters,Education_PhD,Region_Rural,Region_Urban,Political_Affiliation_Conservative,Political_Affiliation_Independent,Political_Affiliation_Liberal
0,0.032787,0.027595,0,0.25,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.754098,0.470945,1,0.50,1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.819672,0.853105,1,0.25,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,0.836066,0.107162,1,0.75,1,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0.688525,0.523825,1,0.50,1,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0


In [73]:
y=dataset['Voting_Intention']

In [74]:
x=dataset.drop('Voting_Intention',axis=1)

In [75]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [76]:
model=LogisticRegression()

In [77]:
model.fit(x_train,y_train)

LogisticRegression()

In [78]:
y_pred=model.predict(x_test)

In [79]:
acc=accuracy_score(y_test,y_pred)
prec=precision_score(y_test,y_pred)
print("accuracy:",acc)
print("precession:",prec)

accuracy: 0.5054945054945055
precession: 0.5540540540540541


In [80]:
print("Precision Score ",prec)
print("Precision Score ",prec)

Precision Score  0.5540540540540541
Precision Score  0.5540540540540541


In [81]:
joblib.dump(model, 'model.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(encoder, 'encoder.pkl')
print("All artifacts saved successfully:")
print("- model.pkl")
print("- scaler.pkl")
print("- encoder.pkl")
print("- target_encoder.pkl")

All artifacts saved successfully:
- model.pkl
- scaler.pkl
- encoder.pkl
- target_encoder.pkl


In [82]:
pip install gradio

In [83]:

# ==========================================
# 2. FEATURE ORDER (EXACT MATCH)
# ==========================================
TRAINING_FEATURES_ORDER = [
    'Age',
    'Income',
    'Campaign_Engagement',
    'Social_Media_Usage',
    'Gender_Female',
    'Gender_Male',
    'Education_Bachelors',
    'Education_High School',
    'Education_Masters',
    'Education_PhD',
    'Region_Rural',
    'Region_Urban',
    'Political_Affiliation_Conservative',
    'Political_Affiliation_Independent',
    'Political_Affiliation_Liberal'
]

# ==========================================
# 3. LOAD MODEL & TARGET ENCODER
# ==========================================
model = joblib.load('model.pkl')

# ==========================================
# 4. PREDICTION FUNCTION
# ==========================================
def predict_voting_intention(
    age, income, campaign_engagement, social_media_usage,
    gender, education, region, political_affiliation
):

    # Gender encoding
    gender_female = 1 if gender == "Female" else 0
    gender_male = 1 if gender == "Male" else 0

    # Education encoding
    edu_map = {
        "Bachelors": [1, 0, 0, 0],
        "High School": [0, 1, 0, 0],
        "Masters": [0, 0, 1, 0],
        "PhD": [0, 0, 0, 1]
    }
    edu_vals = edu_map[education]

    # Region encoding
    region_rural = 1 if region == "Rural" else 0
    region_urban = 1 if region == "Urban" else 0

    # Political affiliation encoding
    pol_map = {
        "Conservative": [1, 0, 0],
        "Independent": [0, 1, 0],
        "Liberal": [0, 0, 1]
    }
    pol_vals = pol_map[political_affiliation]

    # Create input dataframe
    input_data = [[
        age,
        income,
        campaign_engagement,
        social_media_usage,
        gender_female,
        gender_male,
        *edu_vals,
        region_rural,
        region_urban,
        *pol_vals
    ]]

    input_df = pd.DataFrame(input_data, columns=TRAINING_FEATURES_ORDER)

    # Predict
    prediction_index = model.predict(input_df)[0]
    # Map 0 to 'No' and 1 to 'Yes' for better readability
    predicted_class_label = "Yes" if prediction_index == 1 else "No"

    return f"Voting Intention Prediction: {predicted_class_label}"

# ==========================================
# 5. GRADIO INTERFACE
# ==========================================
iface = gr.Interface(
    fn=predict_voting_intention,
    inputs=[
        gr.Slider(0, 1, value=0.5, label="Age (Scaled)"),
        gr.Slider(0, 1, value=0.5, label="Income (Scaled)"),
        gr.Radio([0, 1], label="Campaign Engagement"),
        gr.Slider(0, 1, value=0.5, label="Social Media Usage"),
        gr.Radio(["Female", "Male"], label="Gender"),
        gr.Dropdown(
            ["Bachelors", "High School", "Masters", "PhD"],
            label="Education Level"
        ),
        gr.Radio(["Rural", "Urban"], label="Region"),
        gr.Dropdown(
            ["Conservative", "Independent", "Liberal"],
            label="Political Affiliation"
        )
    ],
    outputs=gr.Textbox(label="Prediction Result"),
    title="Voting Intention Prediction System",
    description="Predict voter intention using demographic, social, and political features."
)

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://48f3cba94db83fa7e9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
